# # SelfStudySession E for Softwaredesign MA 2025 

## E.1 Decorators

### In Python we can use decorators to modify the functionality of a function by wrapping it in another function, you can find an introductions in the following links
- [pythontips](https://book.pythontips.com/en/latest/decorators.html)
- [programiz](https://www.programiz.com/python-programming/decorator)
- [Python 101](https://python101.pythonlibrary.org/chapter25_decorators.html)

Write a decorator *@positive* that checks arguments to be positive and raise an Exception (with *raise Exception("Sorry, not positive")*). Test this implementation with [Exercise D.3](https://kandolfp.github.io/MECH-M-DUAL-1-SWD/sss/sc.html#exr-montecarlopi2) and [Exercise D.4](https://kandolfp.github.io/MECH-M-DUAL-1-SWD/sss/sc.html#exr-pps). We will look at exception handling later.

Can you modify the decorator such that you can give an optional argument specifying the location to check for positivity?

In [21]:
from functools import wraps

def positive(index):
    #is called first with positive(1)
    def decorator(func): 
        # real wrapper
        def wrapper(*args, **kwargs): # 
            if args[index] <= 0:
                raise Exception("Sorry, not positive")
            # need to return the function result
            return func(*args, **kwargs)
        return wrapper
    return decorator


In [14]:
@positive(1)
def testfun(a, b, c):
    return a + b + c
    

# error for
testfun(-1, -1, -1)
# but not for
testfun(-1, 1, -1)

Exception: Sorry, not positive

### D.3 with decorator

In [24]:
import random
import math
import numpy as np


@positive(0)
def in_unit_circle_np(N: int) -> int: #Generates N random points in the unit square and counts how many M are in the unit circle using numpy
    points = np.random.uniform(0, 1, size=(2, N))
    distances_squared = points[0]**2 + points[1]**2
    M = np.sum(distances_squared <= 1)
    return M

@positive(0)
def estimate_pi(N: int, func: callable) -> float: #Estimates pi by using the in_unit_circle function
    M = func(N)
    pi_estimate = 4 * M / N
    return pi_estimate


print(estimate_pi(0, in_unit_circle_np))

Exception: Sorry, not positive

## D.4 with decorator

In [30]:
import numpy as np
import matplotlib.pyplot as plt

class Prey:
    
    def __init__(self, x, alpha=1.0, beta=0.1):
        self.x = float(x)          # prey population
        self._alpha = float(alpha) # natural prey growth rate
        self._beta  = float(beta)  # predation rate coeff
        self._check()

    def _check(self):
        if self.x < 0: raise ValueError("Prey population must be >= 0.")
        if self._alpha <= 0 or self._beta <= 0:
            raise ValueError("alpha, beta must be > 0.")

    def getGrowthRate(self):   return self._alpha
    @positive(1)
    def setGrowthRate(self, v): self._alpha = float(v); self._check()
    def getPredationRate(self): return self._beta
    @positive(1)
    def setPredationRate(self, v): self._beta = float(v); self._check()


class Predator:
    
    def __init__(self, y, gamma=1.0, delta=0.1):
        self.y = float(y)           # predator population
        self._gamma = float(gamma)  # natural predator death rate
        self._delta = float(delta)  # growth per consumed prey
        self._check()

    def _check(self):
        
        if self.y < 0: raise ValueError("Predator population must be >= 0.")
        if self._gamma <= 0 or self._delta <= 0:
            raise ValueError("gamma, delta must be > 0.")

    def getDeathRate(self):     return self._gamma
    @positive(1)
    def setDeathRate(self, v):  self._gamma = float(v); self._check()
    def getConversionRate(self): return self._delta
    @positive(1)
    def setConversionRate(self, v): self._delta = float(v); self._check()


class Propagation:
    def __init__(self, prey: Prey, predator: Predator, dt=0.01):
        self.prey = prey
        self.predator = predator
        self.dt = float(dt)

    def calcDerivatives(self):
        x = self.prey.x
        y = self.predator.y
        a = self.prey.getGrowthRate()
        b = self.prey.getPredationRate()
        g = self.predator.getDeathRate()
        d = self.predator.getConversionRate()

        dx = a*x - b*x*y            # dx/dt
        dy = d*x*y - g*y            # dy/dt
        return dx, dy

    def step(self):
        dx, dy = self.calcDerivatives()
        self.prey.x     = self.prey.x     + self.dt*dx  
        self.predator.y = self.predator.y + self.dt*dy
        # (optional) re-checks
        self.prey._check()
        self.predator._check()

    def simulate(self, steps):
        X = np.empty(steps+1)
        Y = np.empty(steps+1)
        X[0] = self.prey.x
        Y[0] = self.predator.y
        for k in range(steps):
            self.step()
            X[k+1] = self.prey.x
            Y[k+1] = self.predator.y
        return X, Y
    


# Example usage
prey = Prey(x=100, alpha=1.0, beta=0.1)         # alpha = growth, beta = predation
pred = Predator(y=20,  gamma=0.5, delta=0.02)   # gamma = death, delta = conversion

prop = Propagation(prey, pred, dt=0.01)
X, Y = prop.simulate(5000)

# Plot
t = np.arange(len(X)) * prop.dt
plt.plot(t, X, label="Prey (x)")
plt.plot(t, Y, label="Predator (y)")
plt.xlabel("Time")
plt.ylabel("Population")
plt.legend()
plt.tight_layout()
plt.show()

ValueError: Predator population must be >= 0.

## E.2 Pydantic

### In Python the [Pydantic](https://docs.pydantic.dev/latest/) module is a widely used data validation library.


Have a look at [Validators](https://docs.pydantic.dev/latest/concepts/validators/) and see if they can help with [Exercise E.1](https://kandolfp.github.io/MECH-M-DUAL-1-SWD/sss/rs.html#exr-decorators).

### D.3 with pydantic validators

In [35]:
import random
import math
import numpy as np
from pydantic import BaseModel, field_validator

class PositiveInt(BaseModel):
    N: int
    
    @field_validator("N")
    @classmethod
    def check_positive(cls, v):
        if v <= 0:
            raise ValueError("Value must be positive")
        return v

def in_unit_circle_np(N: int) -> int: #Generates N random points in the unit square and counts how many M are in the unit circle using numpy
    #validate N
    cfg = PositiveInt(N=N)
    N = cfg.N

    points = np.random.uniform(0, 1, size=(2, N))
    distances_squared = points[0]**2 + points[1]**2
    M = np.sum(distances_squared <= 1)
    return M


def estimate_pi(N: int, func: callable) -> float: #Estimates pi by using the in_unit_circle function
    #validate N
    cfg = PositiveInt(N=N)
    N = cfg.N

    M = func(N)
    pi_estimate = 4 * M / N
    return pi_estimate


print(estimate_pi(1000000, in_unit_circle_np))

3.144928


## E.3 Define log level via environment variable

### In *Chapter 11* we specified how to customize the logger for a project. Quite often you want to define the log level via an envrionment variable. If the variable *myloglevel=DEBUG* is set, it is used. If nothing is specified the devault value is used.

Implement this for the following snippet and test it accordingly to produce different output.

*Hint*: The *os* module provides a function to get environment variables, [docs](https://docs.python.org/3/library/os.html#os.getenv).

In [40]:
import logging
import os

loglevel_name = os.getenv("LOGLEVEL", "DEBUG").upper()
loglevel = getattr(logging, loglevel_name, logging.DEBUG)

logging.basicConfig(format="%(asctime)s:%(levelname)s: %(message)s",
                    level=loglevel)

numberlist = [-2, -1, 0, 1, 2, "a", 1 / 4]

for number in numberlist:    
    try:
        logging.debug(f"Working on number {number}")
        inverse = 1.0 / number
    except ZeroDivisionError as e:
        logging.error(f"Tried to divide by zero, error is {e}")
    except TypeError:
        logging.warning(f"The list does not only contain numbers")

2025-11-06 18:17:30,936:DEBUG: Working on number -2
2025-11-06 18:17:30,936:DEBUG: Working on number -1
2025-11-06 18:17:30,938:DEBUG: Working on number 0
2025-11-06 18:17:30,938:ERROR: Tried to divide by zero, error is float division by zero
2025-11-06 18:17:30,939:DEBUG: Working on number 1
2025-11-06 18:17:30,939:DEBUG: Working on number 2
2025-11-06 18:17:30,940:DEBUG: Working on number a
2025-11-06 18:17:30,940:WARNING: The list does not only contain numbers
2025-11-06 18:17:30,941:DEBUG: Working on number 0.25


## E.4 Pathlib

### In Python the [pathlib](https://docs.python.org/3/library/pathlib.html) module is a widely used for working with paths and files. Use it to solve the following exercises.

- Create a directory *tmp* with current path of the program. Do not forget to first check if the directory already exists.

    - What would be the alternative with a *try-catch* statement?



In [41]:
from pathlib import Path
from collections import Counter

# Create directory tmp
tmp_dir = Path("tmp")

if not tmp_dir.exists():
    tmp_dir.mkdir()

#Try Catch approach 

try:
    tmp_dir.mkdir()
except FileExistsError:
    pass


- Create the following files with this directory: *[test.txt, file.txt, README.md, random.py]*

In [42]:
filenames = ["test.txt", "file.txt", "README.md", "random.py"]

for filename in filenames:
    file_path = tmp_dir / filename
    try:
        file_path.touch()
    except FileExistsError:
        pass

- Use *collections.Counter* to get a dictionary with the number of files per ending in the directory tmp. The output should look something like this: 

In [ ]:
Counter({'.md': 1, '.txt': 2, '.py': 1})

In [44]:
suffix_cnt = Counter(p.suffix for p in tmp_dir.iterdir())
print(suffix_cnt)

Counter({'.txt': 2, '.py': 1, '.md': 1})


- Find the last modified file in the directory

In [45]:
last_modded = max(tmp_dir.iterdir(), key=lambda p: p.stat().st_mtime)
print(f"Last modified file: {last_modded.name}")

Last modified file: random.py


- Clean up and delete all files al well as the directory

In [46]:
for p in tmp_dir.iterdir():
    if p.is_file():
        p.unlink()

tmp_dir.rmdir()

# Completed as of 06.11.2025 18:42